In [ ]:
import ee
import geemap

# 加载地图
Map = geemap.Map(center=[40,-100], zoom=4)

In [ ]:
# 1.Load ROI
roi = ee.FeatureCollection('your id/assets name/ shapefile name')

# 2.定义去云操作QA60
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

# 3.加载Sentinel-2 影像集
startDate = ''
endDate = ''
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate(startDate, endDate)

    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': -32767,
    'max': 32767,
    'bands': ['B4', 'B3', 'B2'],
}
# 4.转换为Uint16()
image = dataset.filterBounds(roi.geometry()).median().select(['B4', 'B3', 'B2']).multiply(10000).toInt16()

In [ ]:
# 导入必要的库
from tqdm import tqdm

# 定义子图导出函数
def export_subimage_drive(image, roi, output_file_name, output_folder=None, scale=None, file_format=None):

    # 生成子区域列表
    values_list = roi.aggregate_array('id').getInfo()

    # 使用 tqdm 包装循环以显示进度
    for index, value in tqdm(enumerate(values_list), total=len(values_list), desc="Exporting"):

        # 生成子区域矢量图
        test_roi = roi.filterMetadata(name='id', operator='equals', value=value)
        out_img = image.clip(test_roi)

        # 自定义更多导出参数
        region = test_roi  # 导出的区域
        file_format = file_format  # 导出的文件格式

        # 设置配置参数用于输出图像
        task_config = {
            'folder': output_folder,  # 输出 Google Drive 文件夹
            'region': region.geometry(),  # ROI
            'scale': scale,  # 图像分辨率
            'crs': 'EPSG:4326',
            'maxPixels': 1.0E13,
            'fileFormat': file_format
        }
        description = f'{output_file_name + "_" + str(value)}'

        # 导出图像到 Google Drive
        task = ee.batch.Export.image.toDrive(out_img, description, **task_config)
        task.start()


In [ ]:
# 导出


In [ ]:
# 武汉 10m 2021-09-15——2021-10-15
export_subimage_drive(img_wh,roi_wh,output_file_name='wh',output_folder="mmdataset/S2/Wuhan",scale=10, file_format='GEO_TIFF')